# W08 — Pandas: łączenie i agregacja

## Temat: merge, concat, groupby, pivot_table

**Kurs:** Programowanie w języku Python II  
**Politechnika Opolska, WEAiI**  
**Tydzień:** 8

### Efekty uczenia się
Po tym wykładzie osoba studiująca:
1. **Łączy** DataFrame za pomocą `merge()` stosując właściwy typ złączenia
2. **Scala** dane z wielu źródeł za pomocą `pd.concat()`
3. **Agreguje** dane z `groupby()` stosując wiele funkcji i nazwaną agregację
4. **Konstruuje** raporty podsumowujące z `pivot_table()` i `crosstab()`

---

### Schemat relacji tabel

```
klienci (id, imie, miasto, segment)
    |
    | 1:N
    |
zamowienia (id, data, klient_id, produkt_id, ilosc)
    |
    | N:1
    |
produkty (id, nazwa, kategoria, cena)
```

In [ ]:
import pandas as pd
import numpy as np

print(f"pandas {pd.__version__}")
print(f"numpy {np.__version__}")

## Cell 1: Tworzenie danych — sklep TechShop

In [ ]:
# Tabela klientów
klienci = pd.DataFrame({
    'id': range(1, 11),
    'imie': ['Anna Kowalska', 'Piotr Nowak', 'Maria Wiśniewska', 'Jan Kowalczyk',
             'Katarzyna Zielińska', 'Tomasz Lewandowski', 'Agnieszka Wójcik',
             'Michał Kamiński', 'Ewa Kaczmarek', 'Robert Szymański'],
    'miasto': ['Warszawa', 'Kraków', 'Gdańsk', 'Wrocław', 'Poznań',
               'Warszawa', 'Łódź', 'Kraków', 'Gdańsk', 'Wrocław'],
    'segment': ['VIP', 'Standard', 'VIP', 'Standard', 'Premium',
                'Premium', 'Standard', 'VIP', 'Standard', 'Premium']
})

# Tabela produktów
produkty = pd.DataFrame({
    'id': range(1, 9),
    'nazwa': ['Laptop ProX', 'Mysz bezprzewodowa', 'Monitor 27"', 'Klawiatura mechaniczna',
              'Słuchawki BT', 'Webcam HD', 'Pendrive 128GB', 'Hub USB-C'],
    'kategoria': ['Komputery', 'Akcesoria', 'Komputery', 'Akcesoria',
                  'Audio', 'Akcesoria', 'Storage', 'Akcesoria'],
    'cena': [3999.99, 89.99, 1299.99, 249.99, 399.99, 199.99, 49.99, 149.99]
})

# Tabela zamówień
zamowienia = pd.DataFrame({
    'id': range(1, 21),
    'data': pd.to_datetime([
        '2024-01-05', '2024-01-12', '2024-01-15', '2024-01-20',
        '2024-02-03', '2024-02-08', '2024-02-14', '2024-02-19',
        '2024-03-01', '2024-03-07', '2024-03-12', '2024-03-18',
        '2024-04-02', '2024-04-09', '2024-04-15', '2024-04-22',
        '2024-05-05', '2024-05-11', '2024-05-18', '2024-05-25'
    ]),
    'klient_id': [1, 2, 3, 4, 5, 1, 6, 7, 8, 2,
                  3, 9, 10, 1, 4, 5, 6, 7, 8, 9],
    'produkt_id': [1, 2, 3, 4, 5, 6, 1, 2, 3, 7,
                   4, 5, 8, 2, 6, 1, 3, 7, 5, 4],
    'ilosc': [1, 2, 1, 1, 1, 1, 1, 3, 1, 5, 2, 1, 2, 1, 2, 1, 1, 2, 1, 3]
})

print(f"klienci: {len(klienci)} wierszy × {len(klienci.columns)} kolumn")
print(f"produkty: {len(produkty)} wierszy × {len(produkty.columns)} kolumn")
print(f"zamowienia: {len(zamowienia)} wierszy × {len(zamowienia.columns)} kolumn")

In [ ]:
print("KLIENCI:")
print(klienci.to_string(index=False))

In [ ]:
print("PRODUKTY:")
print(produkty.to_string(index=False))

In [ ]:
print("ZAMÓWIENIA (pierwsze 10):")
print(zamowienia.head(10).to_string(index=False))

## Cell 2: Pierwszy merge — zamówienia + klienci

In [ ]:
# Merge zamówień z klientami po kluczu
zam_kl = zamowienia.merge(
    klienci,
    left_on='klient_id',   # klucz w lewej tabeli (zamowienia)
    right_on='id',         # klucz w prawej tabeli (klienci)
    how='inner',           # typ złączenia
    suffixes=('_zam', '_kl')  # sufiks dla zduplikowanych nazw kolumn
)

print(f"Wynik merge: {zam_kl.shape}")
print(f"Kolumny: {list(zam_kl.columns)}")
print()
print(zam_kl[['id_zam', 'data', 'imie', 'miasto', 'segment', 'ilosc']].head(5))

## Cell 3: Cztery typy złączeń — ilustracja

In [ ]:
# Przykład ilustracyjny — 4 typy merge
tabela_A = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'produkt': ['Laptop', 'Monitor', 'Mysz', 'Klawiatura']
})

tabela_B = pd.DataFrame({
    'id': [3, 4, 5, 6],
    'cena': [249.99, 199.99, 89.99, 149.99]
})

print("Tabela A (produkty):", tabela_A['id'].tolist())
print("Tabela B (ceny):", tabela_B['id'].tolist())
print()

# Diagramy Venna:
# A = {1, 2, 3, 4}  B = {3, 4, 5, 6}  Część wspólna = {3, 4}
opisy = {
    'inner': 'tylko wspólne ({3,4}) → przecięcie',
    'left':  'wszystkie z A + dopasowania z B',
    'right': 'wszystkie z B + dopasowania z A',
    'outer': 'wszystkie z A i B → suma'
}

for how, opis in opisy.items():
    result = tabela_A.merge(tabela_B, on='id', how=how)
    print(f"how='{how}' ({opis}): {len(result)} wierszy")
    print(result.to_string(index=False))
    print()

## Cell 4: Łańcuchowy merge z trzech tabel

In [ ]:
# Kompletne złączenie: zamówienia + klienci + produkty
kompletne = (
    zamowienia
    .merge(klienci, left_on='klient_id', right_on='id', suffixes=('_zam', '_kl'))
    .merge(produkty, left_on='produkt_id', right_on='id', suffixes=('', '_prod'))
)

# Obliczenie wartości zamówienia
kompletne['wartosc'] = kompletne['ilosc'] * kompletne['cena']

# Wyświetlenie najważniejszych kolumn
kolumny = ['id_zam', 'data', 'imie', 'miasto', 'segment', 'nazwa', 'kategoria', 'cena', 'ilosc', 'wartosc']
print(f"Kompletna tabela: {kompletne.shape}")
print(kompletne[kolumny].head(8).to_string(index=False))
print(f"\nŁączna wartość sprzedaży: {kompletne['wartosc'].sum():,.2f} zł")

## Cell 5: pd.concat — sklejanie DataFrame

In [ ]:
# Dane sprzedaży z dwóch kwartałów — ta sama struktura
q1 = pd.DataFrame({
    'miesiac': ['Styczeń', 'Luty', 'Marzec'],
    'sprzedaz': [45230, 38920, 52100],
    'zwroty': [1200, 890, 1450]
})

q2 = pd.DataFrame({
    'miesiac': ['Kwiecień', 'Maj', 'Czerwiec'],
    'sprzedaz': [48700, 55200, 62300],
    'zwroty': [1100, 1350, 1800]
})

# Concat — sklej jeden pod drugim
polrocze = pd.concat([q1, q2], ignore_index=True)

print("Q1:")
print(q1)
print("\nQ2:")
print(q2)
print("\nPierwsze półrocze (po concat):")
print(polrocze)
print(f"\nŁączna sprzedaż: {polrocze['sprzedaz'].sum():,} zł")

## Cell 6: concat z kluczami i concat kolumn (axis=1)

In [ ]:
# Concat z kluczami — zachowuje informację o źródle danych
polrocze_z_kwartałem = pd.concat(
    [q1, q2],
    keys=['Q1', 'Q2'],
    names=['Kwartał', 'Nr']
)
print("Z kluczami (MultiIndex):")
print(polrocze_z_kwartałem)
print()

# Concat kolumn (axis=1)
budzet = pd.DataFrame({
    'miesiac': ['Styczeń', 'Luty', 'Marzec'],
    'budzet': [50000, 45000, 55000]
})
q1_z_budzetem = pd.concat([q1, budzet[['budzet']]], axis=1)
q1_z_budzetem['realizacja_pct'] = (q1_z_budzetem['sprzedaz'] / q1_z_budzetem['budzet'] * 100).round(1)
print("Q1 z budżetem i realizacją:")
print(q1_z_budzetem)

---

## Cell 7: groupby — koncepcja split-apply-combine

In [ ]:
# Podstawowy groupby — sprzedaż per miasto
sprzedaz_miasto = kompletne.groupby('miasto')['wartosc'].sum().round(2)
print("Sprzedaż per miasto [PLN]:")
print(sprzedaz_miasto.sort_values(ascending=False))
print(f"\nTyp wyniku: {type(sprzedaz_miasto).__name__}")
print(f"Indeks: {sprzedaz_miasto.index.tolist()}")

## Cell 8: Wiele funkcji agregujących — .agg()

In [ ]:
# Wiele funkcji jednocześnie
stats_segment = kompletne.groupby('segment')['wartosc'].agg(['sum', 'mean', 'count', 'max'])
print("Statystyki per segment klienta:")
print(stats_segment.round(2))

## Cell 9: Nazwana agregacja (Named Aggregation)

In [ ]:
# Nazwana agregacja — czytelne nazwy kolumn wynikowych
# Składnia: nazwa_kolumny=('kolumna_źródłowa', 'funkcja')
raport_segmentow = kompletne.groupby('segment').agg(
    liczba_zamowien=('id_zam', 'count'),
    laczna_wartosc=('wartosc', 'sum'),
    srednia_wartosc=('wartosc', 'mean'),
    max_zamowienie=('wartosc', 'max')
).round(2)

print("Raport segmentów (nazwana agregacja):")
print(raport_segmentow)

## Cell 10: Groupby po wielu kolumnach

In [ ]:
# Grupowanie po dwóch wymiarach
sprzedaz_miasto_kat = (
    kompletne
    .groupby(['miasto', 'kategoria'])['wartosc']
    .sum()
    .round(2)
)
print("Sprzedaż per miasto i kategoria (Series z MultiIndex):")
print(sprzedaz_miasto_kat.head(8))
print()

# unstack() — rozkłada drugi poziom indeksu do kolumn
print("Po unstack() — tabela 2D:")
print(sprzedaz_miasto_kat.unstack(fill_value=0))

## Cell 11: groupby + transform — udział procentowy

In [ ]:
# transform — wartość grupowa w każdym wierszu (zachowuje długość DF)
kompletne['sprzedaz_segmentu'] = kompletne.groupby('segment')['wartosc'].transform('sum')
kompletne['udzial_w_segmencie'] = (
    kompletne['wartosc'] / kompletne['sprzedaz_segmentu'] * 100
).round(1)

print("5 największych zamówień z udziałem w segmencie:")
print(
    kompletne[['imie', 'segment', 'nazwa', 'wartosc', 'sprzedaz_segmentu', 'udzial_w_segmencie']]
    .sort_values('wartosc', ascending=False)
    .head(5)
    .to_string(index=False)
)

---

## Cell 12: pivot_table — tabela przestawna

In [ ]:
# Tabela przestawna: segment × kategoria
pivot = pd.pivot_table(
    kompletne,
    values='wartosc',     # co agregujemy
    index='segment',      # wiersze
    columns='kategoria',  # kolumny
    aggfunc='sum',        # funkcja agregująca
    fill_value=0,         # NaN → 0
    margins=True          # sumy brzegowe
)

print("Sprzedaż [PLN]: Segment × Kategoria")
print(pivot.round(2))

## Cell 13: crosstab — tabela liczebności

In [ ]:
# crosstab — liczy liczbę wystąpień (nie wartości)
ct = pd.crosstab(
    kompletne['segment'],
    kompletne['miasto'],
    margins=True
)
print("Liczba zamówień: Segment × Miasto")
print(ct)
print()

# Normalizacja — procenty per wiersz
ct_pct = pd.crosstab(
    kompletne['segment'],
    kompletne['miasto'],
    normalize='index'
)
print("Udziały procentowe per segment [%]:")
print((ct_pct * 100).round(1))

---

## Aktywność — Raport dla zarządu TechShop

Odpowiedz na trzy pytania biznesowe korzystając z tabeli `kompletne`.

1. Które 3 produkty wygenerowały największy przychód?
2. Jaki jest średni koszyk (średnia wartość zamówienia) per miasto?
3. Zbuduj tabelę przestawną: miesiąc (rows) × kategoria (columns), wartość = suma sprzedaży

*Wskazówka do pyt. 3:* `kompletne['miesiac'] = kompletne['data'].dt.month`

In [ ]:
# Rozwiązanie 1: Top 3 produkty wg przychodu
top3 = kompletne.groupby('nazwa')['wartosc'].sum().nlargest(3).round(2)
print("Top 3 produkty wg przychodu:")
print(top3)

In [ ]:
# Rozwiązanie 2: Średni koszyk per miasto
koszyk = kompletne.groupby('miasto')['wartosc'].mean().round(2).sort_values(ascending=False)
print("Średni koszyk per miasto [PLN]:")
print(koszyk)

In [ ]:
# Rozwiązanie 3: Tabela miesięczna
kompletne['miesiac'] = kompletne['data'].dt.month
pivot_czas = pd.pivot_table(
    kompletne,
    values='wartosc',
    index='miesiac',
    columns='kategoria',
    aggfunc='sum',
    fill_value=0,
    margins=True
)
print("Sprzedaż miesięczna per kategoria [PLN]:")
print(pivot_czas.round(2))

---

## Podsumowanie

| Narzędzie | Kiedy używać | Analogia SQL |
|-----------|-------------|-------------|
| `merge(how='inner')` | Tylko wiersze z parami w obu tabelach | `INNER JOIN` |
| `merge(how='left')` | Wszystkie z lewej + dopasowania z prawej | `LEFT JOIN` |
| `merge(how='outer')` | Wszystkie wiersze z obu tabel | `FULL OUTER JOIN` |
| `pd.concat([df1, df2])` | Sklejanie tabel o tych samych kolumnach | `UNION ALL` |
| `groupby().agg()` | Agregacja po grupach | `GROUP BY` |
| `groupby().transform()` | Wartość grupowa w każdym wierszu | `PARTITION BY` (window) |
| `pivot_table()` | Deklaratywna tabela przestawna | `GROUP BY` + `PIVOT` |
| `crosstab()` | Tabela liczebności / częstości | `GROUP BY` z COUNT |

### Następny wykład — W09: Matplotlib
Dane przygotowane dzisiaj (groupby, pivot) będziemy wizualizować jako wykresy słupkowe, liniowe i kołowe.